In [ ]:
import cv2
from PIL import Image
import urllib.request
from io import BytesIO

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def rnd(a=10, b=20):
    ''' Generate Random Number '''
    return np.random.randint(a, b)

def rect():
    ''' Generate Random Rect '''
    ps = [np.array([0, 0])]
    ps.append(ps[-1] + np.array([rnd(), 0]))
    ps.append(ps[-1] + np.array([0, rnd()]))
    ps.append(ps[-1] + np.array([-rnd(), 0]))
    
    x0 = rnd(a=10, b=100)
    y0 = rnd(a=10, b=100)
    
    x = [e[0] + x0 for e in ps]
    y = [e[1] + y0 for e in ps]
    
    x.append(x[0])
    y.append(y[0])
    
    return x, y
    

In [ ]:
def rgb2str(rgb):
    ''' Convert RGB to String like #FFFFFF'''
    return '#' + ''.join([hex(int(e)).replace('x', '')[-2:] for e in rgb])

def hsv2rgb(hsv):
    ''' Convert Color Array from HSV to RGB '''
    hsv = hsv[np.newaxis, np.newaxis, :].astype(np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR).ravel()

def color_queue(num=15, output_str=True):
    ''' Generate Queue of Colors 
    If output_str is True, the RGB Array will be converted into String like #FFFFFF
    '''
    queue_rgb = []
    queue_hsv = []
    
    threshold = 100**2 + 100**2 + 100**2
    
    while True:
        kwargs = dict(a=10, b=230)
        r0 = rnd(**kwargs)
        r1 = rnd(**kwargs)
        g0 = rnd(**kwargs)
        g1 = rnd(**kwargs)
        b0 = rnd(**kwargs)
        b1 = rnd(**kwargs)
        
        if (r0-r1)**2 + (g0-g1)**2 + (b0-b1)**2 > threshold:
            break
    
    for r, g, b in zip(np.linspace(r0, r1, num), np.linspace(g0, g1, num), np.linspace(b0, b1, num)):
        queue_rgb.append((r, g, b))
        r = r % 180
        queue_hsv.append(hsv2rgb(np.array([r, g, b])))
        
    if output_str:
        return [rgb2str(e) for e in queue_rgb], [rgb2str(e) for e in queue_hsv]
    else:
        return queue_rgb, queue_hsv

In [ ]:
def ravel(d):
    ''' Ravel the 1st and the 2nd Dimensions '''
    s = d.shape
    n = s[0] * s[1]
    d = d.reshape((n, s[2]))
    return d

In [ ]:
url = 'http://localhost:8000/Colors.png'
req = urllib.request.urlopen(url)
im = Image.open(BytesIO(req.read()))
im

In [ ]:
rgb = np.array(im.convert('RGB'))[:, :2000]

uniques = np.unique(rgb[:, :, 2].ravel())

rgb.shape, uniques.shape, uniques

In [ ]:
rgb_data = []
hsv_data = []

rgb_queue, hsv_queue = color_queue(len(uniques), output_str=False)

for rgb_arr, hsv_arr in zip(rgb_queue, hsv_queue):
    x, y = rect()
    rgb_data.append(go.Scatter(x=x, y=y, fillcolor=rgb2str(rgb_arr), fill='toself', line_width=0, opacity=0.5, mode='lines'))
    hsv_data.append(go.Scatter(x=x, y=y, fillcolor=rgb2str(hsv_arr), fill='toself', line_width=0, opacity=0.5, mode='lines'))

fig = make_subplots(rows=2, cols=1, subplot_titles=['RGB', 'HSV'])

for d in rgb_data:
    fig.add_trace(
        d,
        row=1,
        col=1,
    )

for d in hsv_data:
    fig.add_trace(
        d,
        row=2,
        col=1,
    )

fig.update_layout(dict(
        height=800,
        width=800,
    ))
    
fig.show()

In [ ]:
# RawImg
img = Image.fromarray(rgb)
img

In [ ]:
# ReFill using RGB
rgb1 = rgb.copy()
for j in range(1, len(uniques)):
    rgb1[rgb1[:, :, 2] == uniques[j]] = rgb_queue[j]
    
img1 = Image.fromarray(rgb1)
img1

In [ ]:
# ReFill using HSV
rgb2 = rgb.copy()
for j in range(1, len(uniques)):
    rgb2[rgb2[:, :, 2] == uniques[j]] = hsv_queue[j]
    
img2 = Image.fromarray(rgb2)
img2